# generate a target list: see var2df

In [1]:
# 'C[30..80] H[40..300] O[10] N[1] P[1]'
# 'C[10..40] H[20..100] O[2]'
https://www.lfd.uci.edu/~gohlke/molmass/
https://pypi.org/project/molmass/
https://pyteomics.readthedocs.io/en/latest/mass.html
https://github.com/mobiusklein/brainpy

https://stackoverflow.com/questions/53699012/performant-cartesian-product-cross-join-with-pandas

https://stackoverflow.com/questions/13269890/cartesian-product-in-pandas

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Interval.html

# https://stackoverflow.com/questions/21316628/make-a-pandas-multiindex-from-a-product-of-iterables
# https://stackoverflow.com/questions/44452751/pandas-dataframe-rangeindex
# pd.RangeIndex(start=30, stop=80, step=1)

np.arange(f,t) vs range , no range was faster by 10%
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.MultiIndex.from_arrays.html#pandas.MultiIndex.from_arrays

In [5]:
import pandas as pd
import numpy as np
from collections import namedtuple

In [6]:
c = np.arange(30,80+1)
h = np.arange(40,300+1)
o = np.arange(10,10+1)
n = np.arange(1,1+1)
p = np.arange(1,1+1)


In [10]:
# %%timeit
index  = pd.MultiIndex.from_product([c,h,o,n,p], names = ['c','h','o','n','p'])
df = pd.DataFrame(index = index).reset_index()

# calculate the mass ... efficiently

https://towardsdatascience.com/different-ways-to-iterate-over-rows-in-a-pandas-dataframe-performance-comparison-dc0d5dcef8fe
https://stackoverflow.com/questions/16476924/how-to-iterate-over-rows-in-a-dataframe-in-pandas
https://stackoverflow.com/questions/1422149/what-is-vectorization

note the 'at' operator

In [11]:
mass_of = {

}

df['mass'] = 0
for colName in df:
    if colName == 'mass': continue
    colName['mass'] += df[colName] * mass_of[colName]
    


c
h
o
n
p


In [12]:
df['mass'] = 0
df

,c,h,o,n,p,mass
0,30,40,10,1,1,0
1,30,41,10,1,1,0
2,30,42,10,1,1,0
3,30,43,10,1,1,0
4,30,44,10,1,1,0
...,...,...,...,...,...,...
13306,80,296,10,1,1,0
13307,80,297,10,1,1,0
13308,80,298,10,1,1,0
13309,80,299,10,1,1,0
